In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Features relevant to predicting price of a listing

In [2]:
columns = ["Country", "City", "State", "Neighbourhood Cleansed", 
"Host Since", "Host Response Time", "Host Response Rate", "Calculated host listings count",
"Property Type", "Room Type", "Accommodates", "Bedrooms", "Beds", "Bed Type",  "Square Feet", "Cancellation Policy",
"Minimum Nights", "Maximum Nights", "Has Availability", "Availability 30", "Availability 60", "Availability 90", "Availability 365",
"Number of Reviews", "Reviews per Month", "First Review", "Last Review", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness","Review Scores Checkin", "Review Scores Communication", "Review Scores Location","Review Scores Value",
"Features",
"Amenities",
"Price"]

# "Neighbourhood Group Cleansed",
# "Square Feet": 97.56% of entries have NULL values => cannot be used
# "Calendar Updated"
# "Weekly Price", "Monthly Price", "Security Deposit", "Cleaning Fee", "Guests Included", "Extra People",

features_new_host = ["Country", "City", "Neighbourhood Cleansed",
                   "Property Type", "Room Type", "Accommodates","Bedrooms", "Beds", "Bed Type", "Cancellation Policy",
                   "Minimum Nights",
                   "Price"]

features_existing_host = ["Country", "City", "Neighbourhood Cleansed",
                          "Property Type", "Room Type", "Accommodates", "Bedrooms", "Beds", "Bed Type", "Cancellation Policy",
                          "Minimum Nights", "Availability 30", "Availability 60", "Availability 90", "Availability 365",
                          "Number of Reviews", "Reviews per Month", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness","Review Scores Checkin", "Review Scores Communication", "Review Scores Location","Review Scores Value",
                          "Host Since", "Host Response Time", "Host Response Rate", "Calculated host listings count",
                          "Price"]

############################################
########## Feature pre-processing ##########
#### Numerical (Input text fields)
# "Minimum Nights", "Availability 30", "Availability 60", "Availability 90", "Availability 365"
# "Number of Reviews", 
# Review Scores Rating(20-100)
# "Host Since" (num_days = current_date - host_since_date), Host Response Rate(0-100), "Calculated host listings count"
# Price (Convert from local currency to USD)

#### Numerical (Dropdown numerical values)
# Accommodates(1-16), "Bedrooms"(0-10), "Beds"(0-16)
# "Review Scores Accuracy(2-10)", "Review Scores Cleanliness(2-10)","Review Scores Checkin(2-10)", "Review Scores Communication(2-10)", "Review Scores Location(2-10)","Review Scores Value(2-10)"

#### Label encoding (0, 1, 2 .... n_categories-1) (Category dropdowns -> on user select must be assigned numerical value)
# "Country", "City", "Neighbourhood Cleansed"
# "Property Type", "Room Type", "Bed Type", "Cancellation Policy"
# "Host Response Time"
############################################


# TODO
# Host since -> convert to days ----------------------------------- Done
# Features -> Extract features  ----------------------------------- Done
# Amenities -> Extract features ----------------------------------- Done
# Price -> Convert to local currency
# Verify missing features (Listing type etc...) ------------------- Done
# Decide what plots to be shown in exploratory analysis section ---
# Flask vs Flast + React ------------------------------------------ Done (Flask)

In [3]:
df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings.csv", usecols=columns, sep=';')
df = df[columns]
df.head(2)

/Users/nesara/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,City,State,Neighbourhood Cleansed,Host Since,Host Response Time,Host Response Rate,Calculated host listings count,Property Type,Room Type,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Features,Amenities,Price
0,Netherlands,Amsterdam,Noord-Holland,Watergraafsmeer,2017-03-28,NaN,NaN,1.0,Apartment,Entire home/apt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Host Has Profile Pic,"TV,Kitchen,Heating,Washer,Smoke detector,Lapto...",80.0
1,Netherlands,Amsterdam,Noord-Holland,Watergraafsmeer,2014-10-21,within a day,100.0,1.0,House,Entire home/apt,...,94.0,9.0,9.0,10.0,9.0,10.0,9.0,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Wireless Internet,Kitchen,Heating,Family/ki...",195.0


In [4]:
#dfo = df.copy()
len(dfo)

485419

## Data cleaning

### Existing hosts

In [46]:
#df = dfo.copy()
df = dfo[features_existing_host].copy()

enable_separate_features = 0
enable_separate_amenities = 0

########################################################
############ Location Features #############
###### Countries ######## 
# Remove countries with very few entries
exclude_low_count_countries_list = ["0", "Cuba", "Mexico", "Uruguay", "Vanuatu", "Vatican City"]
df = df[~df["Country"].isin(exclude_low_count_countries_list)]
# Remove null (3 entries)
df = df[~pd.isnull(df["Country"])]

###### City ######## 
# Remove cities with less than 10 listings 
# (Total over 4000 cities, 800 odd unique cities with more than 10 listings)
df = df.groupby("City").filter(lambda x: len(x)>10)
# Remove null (451 entries)
df = df[~pd.isnull(df["City"])]


###### Neighbourhood Cleansed ######
# Remove integer Neighbourhood names
df = df[df.apply(lambda x: not x["Neighbourhood Cleansed"].startswith("7"), axis=1)].copy()
# Remove Neighbourhood with less than 10 listings 
# (Total over 2200 cities, 1500 odd unique cities with more than 10 listings)
df = df.groupby("Neighbourhood Cleansed").filter(lambda x: len(x)>10)

# Remove null (1 entry)
df = df[~pd.isnull(df["Neighbourhood Cleansed"])]
#######################################################


####################################################################
############ Basic (Beds, Area, Cancellation) Features #############

###### Property Type #####
# Remove null (11 entries)
df = df[~pd.isnull(df["Property Type"])]
# Remove types with less than 10 entries
df = df.groupby("Property Type").filter(lambda x: len(x)>=10)

###### Room Type #####
# Remove null (1 entries)
df = df[~pd.isnull(df["Room Type"])]

###### Accommodates #####
# Remove null (63 entries)
df = df[~pd.isnull(df["Accommodates"])]
# Accommodate maximum 16 people
df = df[df["Accommodates"] <= 16]

###### Bedrooms #########
# Remove null (597 entries)
df = df[~pd.isnull(df["Bedrooms"])]
# Remove more than 10 bedrooms
df = df[df["Bedrooms"] <= 10]

###### Beds #########
# Remove null (903 entries)
df = df[~pd.isnull(df["Beds"])]
# Remove more than 16 beds (2 entries)
df = df[df["Beds"] <= 16]

###### Bed Type #########
# Remove null (1 entry)
df = df[~pd.isnull(df["Bed Type"])]

###### Cancellation Policy #########
# Remove cancellation policy with low entries (3)
exclude_cancellation_policy_list = ["long_term", "no_refunds"]
df = df[~df["Cancellation Policy"].isin(exclude_cancellation_policy_list)]
# Remove null (2 entries)
df = df[~pd.isnull(df["Cancellation Policy"])]
####################################################################


##########################################################
######### Availability #########

###### Minimum Nights #########
# Remove "Minimum Nights" above 30 except for multiples of 30 (corresponding to months)
df = df[(df["Minimum Nights"]<=31) | (df["Minimum Nights"].isin([60, 90, 120, 180]))]
# Remove null (2 entries)
df = df[~pd.isnull(df["Minimum Nights"])]

###### Availability 30 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 30"])]

###### Availability 60 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 60"])]

###### Availability 90 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 90"])]

###### Availability 365 #########
# Remove null (2 entries)
df = df[~pd.isnull(df["Availability 365"])]
##########################################################


##########################################################
######### Reviews #########

###### Number of Reviews #########
# Remove entries with 0 reviews (Use Model for new host in this case)
df = df[df["Number of Reviews"] > 0]
# Remove null (2 entries)
df = df[~pd.isnull(df["Number of Reviews"])]

###### Reviews per Month #########
#df.at[(df["Number of Reviews"]==0) & (df[pd.isnull("Reviews per Month")]), "Reviews per Month"] = 0
# Remove null (235 entries)
df = df[~pd.isnull(df["Reviews per Month"])]

##### Reviews Scores #############
df = df[~pd.isnull(df["Review Scores Rating"])]
df = df[~pd.isnull(df["Review Scores Accuracy"])]
df = df[~pd.isnull(df["Review Scores Cleanliness"])]
df = df[~pd.isnull(df["Review Scores Checkin"])]
df = df[~pd.isnull(df["Review Scores Communication"])]
df = df[~pd.isnull(df["Review Scores Location"])]
df = df[~pd.isnull(df["Review Scores Value"])]
##########################################################



##########################################################
######### Host Features #########

###### Host Since #########
# Remove null (504 entries)
df = df[~pd.isnull(df["Host Since"])]

# Convert date to number of days till today
today_date = pd.to_datetime(pd.to_datetime("today").date())
df["Host Since"] = pd.to_datetime(df["Host Since"])
df["Host Since Days"] = (today_date - df["Host Since"])
df['Host Since Days'] = df[['Host Since Days']].apply(pd.to_numeric)
df['Host Since Days'] = df['Host Since Days'] / (24*60*60*1e9) #np.timedelta64(1, 'D')
df["Host Since Days"] = df["Host Since Days"].astype(int)

###### Host Response Time #####
# Assign "a few days or more" to entries with NULL values for "Host Response Time"
df.at[pd.isnull(df["Host Response Time"]), "Host Response Time"] = "a few days or more"

###### Host Response Rate #####
# Assign mean(93.4) to entries with NULL values for "Host Response Rate"
df.at[pd.isnull(df["Host Response Rate"]), "Host Response Rate"] = dfo["Host Response Rate"].mean()

###### Calculated host listings count #####
# Assign 1 to entries with NULL values for "Calculated host listings count"
df.at[pd.isnull(df["Calculated host listings count"]), "Calculated host listings count"] = 1
##########################################################



##########################################################
######### Extract Features #########
"""
features_list = []
for i in range(len(df["Features"].unique())):
    if not pd.isnull(df["Features"].unique()[i]):
        features = df["Features"].unique()[i].split(",")
        for ft in features:
            if ft not in features_list:
                features_list.append(ft)
"""

features_list = ['Host Has Profile Pic',
                 'Host Identity Verified',
                 'Is Location Exact',
                 'Instant Bookable',
                 'Host Is Superhost',
                 'Require Guest Phone Verification',
                 'Require Guest Profile Picture',
                 'Requires License']

# Assign 0 to all the above features
for i in range(len(features_list)):
    ft = features_list[i]
    df[ft] = 0


if enable_separate_features == 1:
    # Assign 1 if that feature is present
    for i in range(len(df)):
        idx = df.index[i]
        features = df.loc[idx]["Features"]
        if not pd.isnull(features):
            features = features.split(",")
            for ft in features:
                df.at[idx, ft] = 1
##########################################################



##########################################################
######### Extract Amenities #########
"""
amenities_list = []
for i in range(len(df["Amenities"].unique())):
    if not pd.isnull(df["Amenities"].unique()[i]):
        amenities = df["Amenities"].unique()[i].split(",")
        for ft in amenities:
            if ft not in amenities_list:
                amenities_list.append(ft)
amenities_list
"""
amenities_list = ['TV',
                 'Wireless Internet',
                 'Kitchen',
                 'Heating',
                 'Family/kid friendly',
                 'Washer',
                 'Smoke detector',
                 'Fire extinguisher',
                 'Essentials',
                 'Cable TV',
                 'Internet',
                 'Dryer',
                 'First aid kit',
                 'Safety card',
                 'Shampoo',
                 'Hangers',
                 'Laptop friendly workspace',
                 'Air conditioning',
                 'Breakfast',
                 'Free parking on premises',
                 'Elevator in building',
                 'Buzzer/wireless intercom',
                 'Hair dryer',
                 'Private living room',
                 'Iron',
                 'Wheelchair accessible',
                 'Hot tub',
                 'Carbon monoxide detector',
                 '24-hour check-in',
                 'Pets live on this property',
                 'Dog(s)',
                 'Gym',
                 'Lock on bedroom door',
                 'Private entrance',
                 'Indoor fireplace',
                 'Smoking allowed',
                 'Pets allowed',
                 'Cat(s)',
                 'Self Check-In',
                 'Doorman Entry',
                 'Suitable for events',
                 'Pool',
                 'Lockbox',
                 'Bathtub',
                 'Room-darkening shades',
                 'Game console',
                 'Doorman',
                 'High chair',
                 'Pack ’n Play/travel crib',
                 'Keypad',
                 'Other pet(s)',
                 'Smartlock']

# Assign 0 to all the above amenities
for i in range(len(amenities_list)):
    ft = amenities_list[i]
    df[ft] = 0

    
if enable_separate_amenities == 1:
    # Assign 1 if those amenities are present
    for i in range(len(df)):
        idx = df.index[i]
        amenities = df.loc[idx]["Amenities"]
        if not pd.isnull(amenities):
            amenities = amenities.split(",")
            for ft in amenities:
                df.at[idx, ft] = 1              
##########################################################



##########################################################
######### Price (Output variable) #########


### TODO: Convert to local currency

# Remove null (7954 entries)
df = df[~pd.isnull(df["Price"])]
# Remove 0 (4 entries)
df = df[df["Price"] > 0]

##########################################################

In [47]:
len(df)

337588

In [59]:
df.head(2)

,Country,City,Neighbourhood Cleansed,Property Type,Room Type,Accommodates,Bedrooms,Beds,Bed Type,Cancellation Policy,...,Lockbox,Bathtub,Room-darkening shades,Game console,Doorman,High chair,Pack ’n Play/travel crib,Keypad,Other pet(s),Smartlock
123603,Australia,North Sydney,North Sydney,Apartment,Entire home/apt,4.0,2.0,2.0,Real Bed,moderate,...,0,0,0,0,0,0,0,0,0,0
327368,Spain,Madrid,Casa de Campo,Apartment,Entire home/apt,4.0,2.0,2.0,Real Bed,moderate,...,0,0,0,0,0,0,0,0,0,0


In [48]:
### Price Categories (in USD)
# 1:10-50 2:50-75 3:75-100 4:100-150 5:150-200 6:>200
df["Price Label"] = pd.cut(df["Price"],
                           bins=[0., 50., 75., 100.0, 150., 200., np.inf],
                           labels=[1, 2, 3, 4, 5, 6])

# Distribution of class labels in overall dataset
# df["Price Label"].hist()

### Split Dataset into train and test sets

In [49]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

df.reset_index(inplace=True)
del df["index"]

"""
dfx = df.copy()
del dfx["Price"]
dfy = df["Price"].copy()
x_train, x_test, y_train, y_test = train_test_split(dfx,dfy, test_size=0.1, random_state=42)
"""

# Stratified split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_index in split.split(df, df["Price Label"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

### Univariate, scatter matrix analysis

In [50]:
df_strat_train = strat_train_set.copy()

#from pandas.plotting import scatter_matrix
#attributes = ["Review Scores Rating", "Price"]
#scatter_matrix(df[attributes], figsize=(12, 8))

### Prepare the data for Machine Learning algorithms

In [51]:
# drop labels for training set
df = strat_train_set.drop("Price Label", axis=1) 
df_labels = strat_train_set["Price Label"].copy()

In [77]:
strat_train_set["Price Label"]

123603    6
327368    3
334497    3
42494     1
122021    5
59660     6
251551    6
130785    3
88290     5
318684    3
235877    4
108001    1
136857    1
84146     2
51317     4
214198    1
304496    1
307172    2
286068    6
310253    6
95172     2
89757     1
180278    5
47216     4
35585     4
241618    1
295082    3
33786     3
263437    6
47596     4
         ..
329590    2
146850    6
334137    3
125901    4
4787      6
76522     3
32272     3
157480    1
257203    2
324865    5
175912    6
47568     3
314599    3
267310    1
244291    6
326264    4
219473    4
163630    5
121613    3
153687    3
277049    3
24321     1
147174    4
46036     3
35271     3
221423    1
110504    1
187477    1
306281    3
87283     4
Name: Price Label, Length: 303829, dtype: category
Categories (6, int64): [1 < 2 < 3 < 4 < 5 < 6]

#### Handling missing values

In [ ]:
"""
# Check if there are any rows with NULL values
sample_incomplete_rows = df[df.isnull().any(axis=1)] #.head()
len(sample_incomplete_rows)

# Option 1: Remove rows with NULL values for any subset of columns
# sample_incomplete_rows.dropna(subset=["Bedrooms"])

# Option 2: Remove columns with NULL values
# sample_incomplete_rows.drop("Bedrooms", axis=1)

# Option 3: Fill NULL values with median values of corresponding columns

### Method 1: Manual
# median = df["Bedrooms"].median()
# sample_incomplete_rows["Bedrooms"].fillna(median, inplace=True)

### Method 2: sklearn in-built (Works when all numerical columns NULL is to be replaced with median)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
# Select numerical columns only
df_num = df_xtrain.select_dtypes(include=[np.number])
imputer.fit(df_num)
X = imputer.transform(df_num)
Xtrain = pd.DataFrame(X, columns=df_num.columns,
                          index=df_num.index)
"""

#### Adding extra features 

In [ ]:
"""
from sklearn.preprocessing import FunctionTransformer

def add_extra_features(X, add_bedrooms_per_room=True):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]
    else:
        return np.c_[X, rooms_per_household, population_per_household]

attr_adder = FunctionTransformer(add_extra_features, validate=False,
                                 kw_args={"add_bedrooms_per_room": False})
df_extra_attribs = attr_adder.fit_transform(df.values)
"""

#### Combined pipeline for both numerical and categorical features

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer


##########################################################
############ Pipeline for numerical features #############
# Select numerical columns only
df_num = df.select_dtypes(include=[np.number])
numerical_pipeline = Pipeline([
        #('imputer', SimpleImputer(strategy="median")),
        #('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
    ])
#df_num_tr = numerical_pipeline.fit_transform(df_num)
#df_num_tr
##########################################################


##########################################################
########## Pipeline for categorical features #############
categorical_columns = ["Country", "City", "Neighbourhood Cleansed",
                    "Property Type", "Room Type", "Bed Type", "Cancellation Policy",
                    "Host Response Time"]
#df_cat = df[categorical_columns]

# Encode categorical features as integers
# ordinal_encoder = OrdinalEncoder()
# df_cat_encoded = ordinal_encoder.fit_transform(df_cat)
##########################################################


##########################################################
########## Combined Pipeline for all features ############
numerical_attribs = list(df_num)  
categorical_attribs = categorical_columns

full_pipeline = ColumnTransformer([
        ("numerical", numerical_pipeline, numerical_attribs),
        ("categorical", OrdinalEncoder(), categorical_attribs),
    ])

df_prepared = full_pipeline.fit_transform(df)
##########################################################

In [72]:
df_prepared.shape

(303829, 89)

## Model training and selection

In [81]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

#### Linear Regression

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

lin_reg = LinearRegression()
lin_reg.fit(df_prepared, df_labels)


df_predictions = lin_reg.predict(df_prepared)
lin_mse = mean_squared_error(df_labels, df_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

lin_mae = mean_absolute_error(df_labels, df_predictions)
print(lin_mae)

# Cross Validation
lin_scores = cross_val_score(lin_reg, df_prepared, df_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

0.9546729000895465
0.7601544488960017
Scores: [0.9528555  0.95415046 0.96374648 0.95450051 0.94406339 0.95805711
 0.95433024 0.95305073 0.95644868 0.95680724]
Mean: 0.9548010349274805
Standard deviation: 0.004693009386483896


#### Decision Tree Regressor

In [83]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(df_prepared, df_labels)

df_predictions = tree_reg.predict(df_prepared)
tree_mse = mean_squared_error(df_labels, df_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

# Cross Validation
scores = cross_val_score(tree_reg, df_prepared, df_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

0.0
Scores: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Mean: 0.0
Standard deviation: 0.0


#### Random Forest Regressor

In [85]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# With fixed values of model parameters
forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(df_prepared, df_labels)
df_predictions = forest_reg.predict(df_prepared)
forest_mse = mean_squared_error(df_labels, df_predictions)
forest_rmse = np.sqrt(forest_mse)
print(forest_rmse)


# Cross Validation Scores
forest_scores = cross_val_score(forest_reg, df_prepared, df_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)



# Grid Search (to find best parameters) CV
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]
forest_reg = RandomForestRegressor(random_state=42)
## train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(df_prepared, df_labels)

# Best estimator (hyperparameter combination) found
grid_search.best_params_
grid_search.best_estimator_

# score of each hyperparameter combination tested during the grid search
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.0
Scores: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Mean: 0.0
Standard deviation: 0.0


#### SVM

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(df_prepared, df_labels)
df_predictions = svm_reg.predict(df_prepared)
svm_mse = mean_squared_error(df_labels, df_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse